## Writeup Template

### You can use this file as a template for your writeup if you want to submit it as a markdown file, but feel free to use some other method and submit a pdf if you prefer.

---

**Advanced Lane Finding Project**

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]: ./examples/undistort_output.png "Undistorted"
[image2]: ./test_images/test1.jpg "Road Transformed"
[image3]: ./examples/binary_combo_example.jpg "Binary Example"
[image4]: ./examples/warped_straight_lines.jpg "Warp Example"
[image5]: ./examples/color_fit_lines.jpg "Fit Visual"
[image6]: ./examples/example_output.jpg "Output"
[video1]: ./project_video.mp4 "Video"

## [Rubric](https://review.udacity.com/#!/rubrics/571/view) Points


### Camera Calibration

For this step, I followed the guidelines provided with [Camera Calibration and 3D Reconstruction](https://docs.opencv.org/3.4.3/dc/dbb/tutorial_py_calibration.html) tutorial. The implementation could be found in [solution_calibrate.ipynb](./solution_calibrate.ipynb). It provides [calibration](./solution_calibrate.ipynb#Calibrate-camera), [debug/slideshow](./solution_calibrate.ipynb#Debug-calibration) and [sample generation](./solution_calibrate.ipynb#Generate-sample) (used by this writeup) as well as functions required by those step. Calibration has following pipeline...

#### 1) findPatterns,

Finds object and image points for a given image set and chessboard pattern (nx & ny).

Chessboard was kept stationary in the given calibration samples. So, I believe that is why object points are constant in-between pattern finding and all object-points (I am still trying to wrap my head around this idea). The variable is the camera and our perception in the 2D space (image-points).

This function returns image and object point pairs required for calibrating the camera.

#### 2) calibrate,

Calculates camera matrix and distortion coefficient for the given image and patterns (object & image points). This method could be further improved with the `getOptimalNewCameraMatrix` function provided by opencv.

#### 3) saveCalibrationData,

Saves calculated camera matrix and distortion coefficient to a file using pickle. This will allow us to reuse the calibration data in other work notebooks.

**Sample input**
![Sample input](./solution_output/calibration_images/input00.jpg)
**Sample output**
![Sample output](./solution_output/calibration_images/output00.jpg)


### Pipeline (single images)

#### 1. Provide an example of a distortion-corrected image.

To demonstrate this step, I will describe how I apply the distortion correction to one of the test images like this one:
![Smaple distortion-corrected image](./solution_output/lane_detection_images/00-1-input.jpg)

#### 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.

Binary image generation is achieved by combining binary gradient and HLS saturation channel applied to undistorted image. It has following pipeline ...

##### 1) undistord, 

Undistords input image based on pre-calculated camera matrix and distortion coefficients.

![Smaple distortion-corrected image](./solution_output/lane_detection_images/00-2-step-binary-input.jpg)

##### 2a) gradientBinary,

Generates a binary image based on given threshold applied to 8-bit sobel image. It generates a binary \[0, 1\] image as an output. The processing involves ...

 - Convert BGR (3-channel) image to grayscale (1-channel)
 - Calculate sobel on x-axis
 - Take absolute value of sobel output
 - Convert absolute sobel into 8-bit by normalizing it between \[0,255\]
 - Apply a binary threshold based on given upper & lower bounds. The values in-between the given boundaries are marked as 1
 
##### 2b) saturationBinary,

Generates a binary image based on given threshold applied to saturation channel of HLS image. It generates a binary \[0, 1\] image as an output. The processing involves ...

 - Convert BGR image to HLS
 - Extract saturation channel (1-channel)
 - Apply a binary threshold based on given upper & lower bounds. The values in-between the given boundaries are marked as 1
 
##### 3) Combine image with bitwise `or` logic

Generates a binary image by combining images generated at steps above (2a & 2b) with using bitwise `or`


Here's an example of my output for this step.  (note: this is not actually from one of the test images)

![Sample binary step output](./solution_output/lane_detection_images/00-5-output-step-binary-combined.jpg)

#### 3. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.

Prior to perspective transform we remove the activated pixels positioned outside the `state.roiVertices` using `removeEnvironmentNoise` function. It helps us to get rid of (noisy) data outside the estimated lane region.

Perspective transform from camera to birds eye view is achieved with the help of `warp` function. It warps perspective using `cv2.warpPerspective` with the help of transformation matrix calculated with `cv2.getPerspectiveTransform`. Both cameraLaneVertices and birdsEyeLaneVertices are calculated once using following logic ...

```python
cameraLaneVertices = np.float32([
    [width * 0.48, height * 0.6], # Top left
    [width * 0.00, height * 0.96], # Bottom left
    [width * 1.00, height * 0.96], # Bottom right
    [width * 0.52, height * 0.6], # Top right
])
birdsEyeLaneVertices = np.float32([
    [width * 0.25, 0], # Top left
    [width * 0.25, height], # Bottom left
    [width * 0.75, height], # Bottom right
    [width * 0.75, 0], # Top right
])
```

This resulted in the following source and destination points:

| Camera        | Birds-eye     | 
|:-------------:|:-------------:| 
| 607 , 439     | 320, 0        | 
| 243 , 684     | 320, 720      |
| 1036, 684     | 960, 720      |
| 672 , 439     | 960, 0        |

I verified that my perspective transform was working as expected by drawing the `src` and `dst` points onto a test image and its warped counterpart to verify that the lines appear parallel in the warped image.

![Camera image](./solution_output/lane_detection_images/00-6-input-step-perspective.jpg)

![Birds-eye image](./solution_output/lane_detection_images/00-7-output-step-perspective.jpg)

#### 4. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

Lane finding is achived in 2 different ways. If we have a trace of previous state of the lane (i.e., 2nd order polynomial), a lookup happens on the range of that lane. If lane state is unknown, we do first search for left and right lane bases. If we could find a base/clue for a starting position, code climbs up using sliding windows until we reach image boundaries.

##### 1) Lane finding with a prior,

It is done with `fitPolynomialPrior`. Marks and collects all the pixel coordinates within the range of prior 2d polynomials for both left (i.e., leftFit) and right (i.e., rightFit). Then, generates a new polynomial for both sides out of those pixels.

##### 2) Lane finding with a sliding window,

It is done with a combination of `findHorizontalLaneBases` and `fitPolynomialSlidingWindow`. Search starts with finding lane start/base points for left and right. It looks into following criterias...

 - Given image needs to have at least 2 separate areas with dense activated pixels
 - Minimum distance between lanes needs to be bigger than `state.minLaneDistance`
 - Minimum density of the activated pixels needs to be bigger than given threshold. e.g., 10 pixels

Finding the densest activated pixels are done by `__binaryImageActivationHistogram`. It sums up all the pixel values [0 & 1] on vertical axis. This generates a 1-D array that we will call histogram (of activated pixels). Yet, that wasn't sufficient to mark the peak points. Generated histogram has many peak points that creates noisy spikes and makes it harder to figure out the peak point. To prevent that, code calculates a rolling average for each pixel. This was quite a nice touch to smoothen all the rough edges in the histogram and reduce the number of peak points in the generated output.

Peak points are identified with `__findPeakPoints` function. It filters out all the pixels that is adjacent to a higher pixel within given reach/lookupRange or threshold/density.

Lane bases are selected among those peak points. If found, it outputs densest peak point with `state.minLaneDistance` to another peak point along with that other peak point

2nd order polynomial kinda like this:

![alt text][image5]

#### 5. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

I did this in lines # through # in my code in [solution_calibrate.ipynb](./solution.ipynb#Lane-detection-functions) `calculateCurvature` function

#### 6. Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

I implemented this step in lines # through # in my code in `yet_another_file.py` in the function `map_lane()`.  Here is an example of my result on a test image:

![alt text][image6]

---

### Pipeline (video)

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!).

Here's a [link to my video result](./solution_output/lane_detection_videos/project_video.mp4)

---

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

Here I'll talk about the approach I took, what techniques I used, what worked and why, where the pipeline might fail and how I might improve it if I were going to pursue this project further.  


##### Getting the right warp vertices was the key point in finding lanes. 

I started with the less accurate vertices that increased the failure rate for the rest of the functions

##### Histogram peaks

Lane pixel histogram had high variance and contained mostly more than 2 peak points. e.g., looking into a pixel and its neighbours might seem like it is a peak, yet the there is actually a higher peak adjacent to that point. Taking rolling average helped me to smoothen the edges for false-positive peaks.

##### Lane curvature and direction

My code doesn't contain a logic for checking detected lanes have similar curvature and direction. That leads to false positives from time to time.


##### Cutting sliding window detection for a lane once it reaches image boundary

While coding sliding windows, I noticed the detected polynomials for a lane results with a false positive, whenever its pixels goes out of image bound. With some debugging, I noticed that sliding window should terminate its detection for a lane, once it reaches to a boundary. Otherwise, it keeps on searching upwards. It is highly likely, it would detect other lane pixels if there is a lane has a curvature


##### My code is not smoothing detected lanes based on n-prior detection

Assuming lanes are less likely to change their state suddenly, I could smooth out the detected lane by taking average of previous n (e.g., 3) detected lane pixels


##### My code is not resistant to bad frames

I am not maintaining a lane detection history and resetting the lane state whenever I fail to detect them. It makes me weak towards bad frames (e.g., sudden shadow, bright light, camera failure, etc...). I could maintain previous n states and use them as a temporary back-up to lane state


##### I still couldn't wrap my head around the idea of using real world metric conversion during parabola calculation. This method is used in `Measuring Curvature II` section
